# Running a Federated Cycle with Synergos

In a federated learning system, there are many contributory participants, known as Worker nodes, which receive a global model to train on, with their own local dataset. The dataset does not leave the individual Worker nodes at any point, and remains private to the node.

The job to synchronize, orchestrate and initiate an federated learning cycle, falls on a Trusted Third Party (TTP). The TTP pushes out the global model architecture and parameters for the individual nodes to train on, calling upon the required data, based on tags, e.g "training", which points to relevant data on the individual nodes. At no point does the TTP receive, copy or access the Worker nodes' local datasets.

![Simulated Synergos Plus Grid](../../docs/images/synplus_setup.png "A simple Synergos Plus setup")

This tutorial aims to give you an understanding of how to use the synergos package to run a full federated learning cycle on a `Synergos Plus` grid.

In a `Synergos Plus` grid, you have access to a suite of quality-of-life addons that can help facilitate distributed operations. Such components include centralized logging via [Synergos Logger](https://github.com/aimakerspace/synergos_logger), artifact management via [Synergos MLOps](https://github.com/aimakerspace/synergos_mlops), with more component support in the works.

In this tutorial, you will go through the steps required by each participant (TTP and Worker), by simulating each of them locally with docker containers. Specifically, we will simulate a TTP and 2 Workers. 

At the end of this, we will have:
- Connected the participants
- Trained the model
- Evaluate the model

## About the Dataset and Task

The dataset used in this notebook is on heart disease tabular data, comprising 14 predictor features, and 1 target feature for a binary class. The dataset is available in the same directory as this notebook. Within the dataset directory, `data1` is for Worker 1 and `data2` is for Worker 2. The task to be carried out will be a binary classifcation.

The dataset we have provided is a processed subset of the [original Heart Disease dataset](https://archive.ics.uci.edu/ml/datasets/heart+disease).

**Reference:**
- *Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.*

## Initiating the docker containers

Before we begin, we have to start the docker containers.

### A. Initialization via `Synergos Simulator`

In `Synergos Simulator`, a sandboxed environment has been created for you!

By running:

    `docker-compose -f docker-compose-synplus.yml up --build`
    
the following components will be started:
- TTP (Basic)
- Worker_1
- Worker_2
- Synergos UI
- Synergos Logger
- Synergos MLOps

Refer to [this](https://github.com/aimakerspace/synergos_simulator) for all the pre-allocated host & port mappings. 

### B. Manual Initialization

Firstly, pull the required docker images with the following commands:

1. Synergos TTP (Basic): 

    `docker pull gcr.io/synergos-aisg/synergos_ttp:v0.1.0`

2. Synergos Worker: 

    `docker pull gcr.io/synergos-aisg/synergos_worker:v0.1.0`
    
3. Synergos MLOps:

    `docker pull gcr.io/synergos-aisg/synergos_mlops:v0.1.0`

Next, in <u>separate</u> CLI terminals, run the following command(s):

**Note: For Windows users, it is advisable to use powershell or command prompt based interfaces**

**TTP**

```
docker run 
    -p 5000:5000 
    -p 8020:8020
    -v <directory heart_disease/ttp_outputs>:/orchestrator/outputs
    -v <directory heart_disease/ttp_data>:/orchestrator/data
    -v <directory heart_disease/mlflow>:/mlflow
    --name ttp 
    gcr.io/synergos-aisg/synergos_ttp:v0.1.0 
        --id ttp 
        --logging_variant graylog <IP Synergos Logger> <TTP port>
```

**Worker 1**

```
docker run 
    -p 5001:5000 
    -p 8021:8020
    -v <directory heart_disease/data1>:/worker/data 
    -v <directory heart_disease/outputs_1>:/worker/outputs 
    --name worker_1 
    gcr.io/synergos-aisg/synergos_worker:v0.1.0 
        --id worker_1 
        --logging_variant graylog <IP Synergos Logger> <Worker port>
```

**Worker 2**

```
docker run 
    -p 5002:5000 
    -p 8022:8020
    -v <directory heart_disease/data2>:/worker/data 
    -v <directory heart_disease/outputs_2>:/worker/outputs 
    --name worker_2 
    gcr.io/synergos-aisg/synergos_worker:v0.1.0
        --id worker_2 
        --logging_variant graylog <IP Synergos Logger> <Worker port>
```

**Synergos MLOps**

```
docker run --rm 
    -p 5500:5500 
    -v /path/to/mlflow_test/:/mlflow    # <-- IMPT! Same as orchestrator's
    --name synmlops 
    gcr.io/synergos-aisg/synergos_mlops:v0.1.0
```

**Synergos UI**

- Refer to these [instructions](https://github.com/aimakerspace/synergos_ui) to deploy `Synergos UI`.

**Synergos Logger**

- Refer to these [instructions](https://github.com/aimakerspace/synergos_logger) to deploy `Synergos Logger`.


Once ready, for each terminal, you should see a REST server running on http://0.0.0.0:5000 of the container.

You are now ready for the next step.

## Configurations

### A. Configuring `Synergos Simulator`

All hosts & ports have already been pre-allocated!

Refer to [this](https://github.com/aimakerspace/synergos_simulator) for all the pre-allocated host & port mappings.

### B. Configuring your manual setup

In a new terminal, run `docker inspect bridge` and find the IPv4Address for each container. Ideally, the containers should have the following addresses:
- ttp address: `172.17.0.2`
- worker_1 address: `172.17.0.3`
- worker_2 address: `172.17.0.4`
- UI address: `172.17.0.5`
- Logger address: `172.17.0.8`
- MLOps address: `172.17.0.9`

If not, just note the relevant IP addresses for each docker container.

Run the following cells below.

**Note: For Windows users, `host` should be Docker Desktop VM's IP. Follow [this](https://stackoverflow.com/questions/58073936/how-to-get-ip-address-of-docker-desktop-vm) on instructions to find IP**

In [1]:
from synergos import Driver

host = "172.19.0.2"
port = 5000

# Initiate Driver
driver = Driver(host=host, port=port)

## Phase 1: Registration

Submitting Orchestrator & Participant metadata

#### 1A. Orchestrator creates a collaboration

In [2]:
collab_task = driver.collaborations

collab_task.configure_logger(
    host="172.19.0.10", 
    port=9000, 
    sysmetrics_port=9100, 
    director_port=9200, 
    ttp_port=9300, 
    worker_port=9400, 
    ui_port=9000, 
    secure=False
)

collab_task.configure_mlops( 
    host="172.19.0.11", 
    port=5500, 
    ui_port=5500, 
    secure=False
)
collab_task.create('heart_disease_synplus_collaboration')

{'data': {'doc_id': '1',
  'kind': 'Collaboration',
  'key': {'collab_id': 'heart_disease_synplus_collaboration'},
  'relations': {'Project': [],
   'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []},
  'logs': {'host': '172.19.0.10',
   'ports': {'sysmetrics': 9100,
    'director': 9200,
    'ttp': 9300,
    'worker': 9400,
    'main': 9000,
    'ui': 9000},
   'secure': False},
  'mlops': {'host': '172.19.0.11',
   'ports': {'main': 5500, 'ui': 5500},
   'secure': False}},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'collaborations.post',
 'params': {}}

#### 1B. Orchestrator creates a project

In [3]:
driver.projects.create(
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    action="classify",
    incentives={
        'tier_1': [],
        'tier_2': [],
    }
)

{'data': {'doc_id': '1',
  'kind': 'Project',
  'key': {'collab_id': 'heart_disease_synplus_collaboration',
   'project_id': 'heart_disease_synplus_project'},
  'relations': {'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []},
  'action': 'classify',
  'incentives': {'tier_2': [], 'tier_1': []}},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'projects.post',
 'params': {'collab_id': 'heart_disease_synplus_collaboration'}}

#### 1C. Orchestrator creates an experiment

In [4]:
driver.experiments.create(
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    expt_id="heart_disease_synplus_experiment",
    model=[
        {
            "activation": "sigmoid",
            "is_input": True,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 13,
                "out_features": 1 
            }
        }
    ]
)

{'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'experiments.post',
 'params': {'collab_id': 'heart_disease_synplus_collaboration',
  'project_id': 'heart_disease_synplus_project'},
 'data': {'created_at': '2021-09-16 00:18:24 N',
  'key': {'collab_id': 'heart_disease_synplus_collaboration',
   'expt_id': 'heart_disease_synplus_experiment',
   'project_id': 'heart_disease_synplus_project'},
  'model': [{'activation': 'sigmoid',
    'is_input': True,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 13, 'out_features': 1}}],
  'relations': {'Run': [], 'Model': [], 'Validation': [], 'Prediction': []},
  'doc_id': 1,
  'kind': 'Experiment'}}

#### 1D. Orchestrator creates a run

In [5]:
driver.runs.create(
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    expt_id="heart_disease_synplus_experiment",
    run_id="heart_disease_synplus_run",
    rounds=2, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="NLLLoss"
)

{'data': {'doc_id': '1',
  'kind': 'Run',
  'key': {'collab_id': 'heart_disease_synplus_collaboration',
   'project_id': 'heart_disease_synplus_project',
   'expt_id': 'heart_disease_synplus_experiment',
   'run_id': 'heart_disease_synplus_run'},
  'relations': {'Model': [], 'Validation': [], 'Prediction': []},
  'rounds': 2,
  'epochs': 1,
  'lr': 0.001,
  'lr_decay': 0.1,
  'weight_decay': 0.0,
  'seed': 42,
  'precision_fractional': 5,
  'mu': 0.1,
  'l1_lambda': 0.0,
  'l2_lambda': 0.0,
  'base_lr': 0.0005,
  'max_lr': 0.005,
  'patience': 10,
  'delta': 0.0},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'runs.post',
 'params': {'collab_id': 'heart_disease_synplus_collaboration',
  'project_id': 'heart_disease_synplus_project',
  'expt_id': 'heart_disease_synplus_experiment'}}

#### 1E. Participants registers their servers' configurations and roles

In [7]:
participant_resp_1 = driver.participants.create(
    participant_id="worker_1",
)

display(participant_resp_1)

participant_resp_2 = driver.participants.create(
    participant_id="worker_2",
)

display(participant_resp_2)

{'data': {'doc_id': '1',
  'kind': 'Participant',
  'key': {'participant_id': 'worker_1'},
  'relations': {'Registration': [{'doc_id': '1',
     'kind': 'Registration',
     'key': {'collab_id': 'heart_disease_synplus_collaboration',
      'project_id': 'heart_disease_synplus_project',
      'participant_id': 'worker_1'},
     'collaboration': {'catalogue': {},
      'logs': {},
      'meter': {},
      'mlops': {},
      'mq': {}},
     'project': {'action': None, 'incentives': {}, 'start_at': None},
     'participant': {'id': None,
      'category': [],
      'summary': None,
      'phone': None,
      'email': None,
      'socials': {}},
     'role': 'host',
     'n_count': 1,
     'node_0': {'host': '172.19.0.3',
      'port': 8020,
      'log_msgs': True,
      'verbose': True,
      'f_port': 5000}}],
   'Tag': []},
  'id': 'worker_1'},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'participants.post',
 'params': {}}

{'data': {'doc_id': '2',
  'kind': 'Participant',
  'key': {'participant_id': 'worker_2'},
  'relations': {'Registration': [], 'Tag': []},
  'id': 'worker_2'},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'participants.post',
 'params': {}}

In [8]:
registration_task = driver.registrations

# Add and register worker_1 node
registration_task.add_node(
    host='172.19.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

registration_task.create(
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    participant_id="worker_1",
    role="host"
)

{'data': {'doc_id': '1',
  'kind': 'Registration',
  'key': {'collab_id': 'heart_disease_synplus_collaboration',
   'project_id': 'heart_disease_synplus_project',
   'participant_id': 'worker_1'},
  'relations': {'Tag': []},
  'collaboration': {'catalogue': {},
   'logs': {'host': '172.19.0.10',
    'ports': {'sysmetrics': 9100,
     'director': 9200,
     'ttp': 9300,
     'worker': 9400,
     'main': 9000,
     'ui': 9000},
    'secure': False},
   'meter': {},
   'mlops': {'host': '172.19.0.11',
    'ports': {'main': 5500, 'ui': 5500},
    'secure': False},
   'mq': {}},
  'project': {'action': 'classify',
   'incentives': {'tier_2': [], 'tier_1': []},
   'start_at': None},
  'participant': {'id': 'worker_1',
   'category': [],
   'summary': None,
   'phone': None,
   'email': None,
   'socials': {}},
  'role': 'host',
  'n_count': 1,
  'node_0': {'host': '172.19.0.3',
   'port': 8020,
   'log_msgs': True,
   'verbose': True,
   'f_port': 5000}},
 'apiVersion': '0.1.0',
 'success': 

In [9]:
registration_task = driver.registrations

# Add and register worker_2 node
registration_task.add_node(
    host='172.19.0.4',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

registration_task.create(
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    participant_id="worker_2",
    role="guest"
)

{'data': {'doc_id': '2',
  'kind': 'Registration',
  'key': {'collab_id': 'heart_disease_synplus_collaboration',
   'project_id': 'heart_disease_synplus_project',
   'participant_id': 'worker_2'},
  'relations': {'Tag': []},
  'collaboration': {'catalogue': {},
   'logs': {'host': '172.19.0.10',
    'ports': {'sysmetrics': 9100,
     'director': 9200,
     'ttp': 9300,
     'worker': 9400,
     'main': 9000,
     'ui': 9000},
    'secure': False},
   'meter': {},
   'mlops': {'host': '172.19.0.11',
    'ports': {'main': 5500, 'ui': 5500},
    'secure': False},
   'mq': {}},
  'project': {'action': 'classify',
   'incentives': {'tier_2': [], 'tier_1': []},
   'start_at': None},
  'participant': {'id': 'worker_2',
   'category': [],
   'summary': None,
   'phone': None,
   'email': None,
   'socials': {}},
  'role': 'guest',
  'n_count': 1,
  'node_0': {'host': '172.19.0.4',
   'port': 8020,
   'log_msgs': True,
   'verbose': True,
   'f_port': 5000}},
 'apiVersion': '0.1.0',
 'success':

#### 1F. Participants registers their tags for a specific project

In [10]:
# Worker 1 declares their data tags
driver.tags.create(
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    participant_id="worker_1",
    train=[["heart_disease", "dataset", "data1", "iid_1"]],
    evaluate=[["heart_disease", "dataset", "data1", "non_iid_1"]]
)

{'data': {'doc_id': '1',
  'kind': 'Tag',
  'key': {'collab_id': 'heart_disease_synplus_collaboration',
   'project_id': 'heart_disease_synplus_project',
   'participant_id': 'worker_1'},
  'train': [['heart_disease', 'dataset', 'data1', 'iid_1']],
  'evaluate': [['heart_disease', 'dataset', 'data1', 'non_iid_1']],
  'predict': []},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'tag.post',
 'params': {'collab_id': 'heart_disease_synplus_collaboration',
  'project_id': 'heart_disease_synplus_project',
  'participant_id': 'worker_1'}}

In [11]:
# Worker 2 declares their data tags
driver.tags.create(
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    participant_id="worker_2",
    train=[["heart_disease", "dataset", "data2", "iid_2"]],
    evaluate=[["heart_disease", "dataset", "data2", "non_iid_2"]]
)

{'data': {'doc_id': '2',
  'kind': 'Tag',
  'key': {'collab_id': 'heart_disease_synplus_collaboration',
   'project_id': 'heart_disease_synplus_project',
   'participant_id': 'worker_2'},
  'train': [['heart_disease', 'dataset', 'data2', 'iid_2']],
  'evaluate': [['heart_disease', 'dataset', 'data2', 'non_iid_2']],
  'predict': []},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'tag.post',
 'params': {'collab_id': 'heart_disease_synplus_collaboration',
  'project_id': 'heart_disease_synplus_project',
  'participant_id': 'worker_2'}}

## Phase 2: 
Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [12]:
driver.alignments.create(
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    verbose=False,
    log_msg=False
)

{'data': [{'doc_id': '1',
   'kind': 'Alignment',
   'key': {'collab_id': 'heart_disease_synplus_collaboration',
    'project_id': 'heart_disease_synplus_project',
    'participant_id': 'worker_1'},
   'train': {'X': [], 'y': []},
   'evaluate': {'X': [], 'y': []},
   'predict': {'X': [], 'y': []}},
  {'doc_id': '2',
   'kind': 'Alignment',
   'key': {'collab_id': 'heart_disease_synplus_collaboration',
    'project_id': 'heart_disease_synplus_project',
    'participant_id': 'worker_2'},
   'train': {'X': [], 'y': []},
   'evaluate': {'X': [], 'y': []},
   'predict': {'X': [], 'y': []}}],
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'alignments.post',
 'params': {'collab_id': 'heart_disease_synplus_collaboration',
  'project_id': 'heart_disease_synplus_project'}}

#### 2B. Trigger training across the federated grid

In [13]:
model_resp = driver.models.create(
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    expt_id="heart_disease_synplus_experiment",
    run_id="heart_disease_synplus_run",
    log_msg=False,
    verbose=False
)
display(model_resp)

{'data': [{'doc_id': '1',
   'kind': 'Model',
   'key': {'project_id': 'heart_disease_synplus_project',
    'expt_id': 'heart_disease_synplus_experiment',
    'run_id': 'heart_disease_synplus_run'},
   'global': {'origin': 'ttp',
    'path': '/orchestrator/outputs/heart_disease_synplus_collaboration/heart_disease_synplus_project/heart_disease_synplus_experiment/heart_disease_synplus_run/global_model.pt',
    'loss_history': '/orchestrator/outputs/heart_disease_synplus_collaboration/heart_disease_synplus_project/heart_disease_synplus_experiment/heart_disease_synplus_run/global_loss_history.json'},
   'local_2': {'origin': 'worker_2',
    'path': '/orchestrator/outputs/heart_disease_synplus_collaboration/heart_disease_synplus_project/heart_disease_synplus_experiment/heart_disease_synplus_run/local_model_worker_2.pt',
    'loss_history': '/orchestrator/outputs/heart_disease_synplus_collaboration/heart_disease_synplus_project/heart_disease_synplus_experiment/heart_disease_synplus_run/local

## Phase 3: EVALUATE 
Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [14]:
# Orchestrator performs post-mortem validation
driver.validations.create(
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    expt_id="heart_disease_synplus_experiment",
    run_id="heart_disease_synplus_run",
    log_msg=False,
    verbose=False
)

{'data': [{'doc_id': '1',
   'kind': 'Validation',
   'key': {'participant_id': 'worker_1',
    'collab_id': 'heart_disease_synplus_collaboration',
    'project_id': 'heart_disease_synplus_project',
    'expt_id': 'heart_disease_synplus_experiment',
    'run_id': 'heart_disease_synplus_run'},
   'evaluate': {'statistics': {'accuracy': [0.5508474576271186,
      0.7784503631961259,
      0.851089588377724,
      0.15375302663438256,
      0.9539951573849879],
     'roc_auc_score': [0.5818903468499156,
      0.3891155980844302,
      0.5190814263394908,
      0.49965769593956566,
      0.5726355864279989],
     'pr_auc_score': [0.5427311556032566,
      0.16643826569934234,
      0.16875920008283063,
      0.1711121039497303,
      0.06824169697771335],
     'f_score': [0.0, 0.01081081081081081, 0.016, 0.2508038585209003, 0.0],
     'TPRs': [0.0, 0.005780346820809248, 0.008064516129032258, 0.975, 0.0],
     'TNRs': [1.0, 0.9831546707503829, 1.0, 0.014164305949008499, 1.0],
     'PPVs': [

#### 3B. Perform prediction(s) of combination(s)

In [15]:
# Worker 1 requests for inferences
driver.predictions.create(
    tags={"heart_disease_synplus_project": [["heart_disease", "dataset", "data1", "non_iid_1"]]},
    participant_id="worker_1",
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    expt_id="heart_disease_synplus_experiment",
    run_id="heart_disease_synplus_run",
)

{'data': [{'doc_id': '1',
   'kind': 'Prediction',
   'key': {'participant_id': 'worker_1',
    'project_id': 'heart_disease_synplus_project',
    'expt_id': 'heart_disease_synplus_experiment',
    'run_id': 'heart_disease_synplus_run'},
   'predict': {'statistics': {'accuracy': [0.5508474576271186,
      0.7784503631961259,
      0.851089588377724,
      0.15375302663438256,
      0.9539951573849879],
     'roc_auc_score': [0.5818903468499156,
      0.3891155980844302,
      0.5190814263394908,
      0.49965769593956566,
      0.5726355864279989],
     'pr_auc_score': [0.5427311556032566,
      0.16643826569934234,
      0.16875920008283063,
      0.1711121039497303,
      0.06824169697771335],
     'f_score': [0.0, 0.01081081081081081, 0.016, 0.2508038585209003, 0.0],
     'TPRs': [0.0, 0.005780346820809248, 0.008064516129032258, 0.975, 0.0],
     'TNRs': [1.0, 0.9831546707503829, 1.0, 0.014164305949008499, 1.0],
     'PPVs': [0.0, 0.08333333333333333, 1.0, 0.14391143911439114, 0.0],

In [16]:
# Worker 2 requests for inferences
driver.predictions.create(
    tags={"heart_disease_synplus_project": [["heart_disease", "dataset", "data2", "non_iid_2"]]},
    participant_id="worker_2",
    collab_id="heart_disease_synplus_collaboration",
    project_id="heart_disease_synplus_project",
    expt_id="heart_disease_synplus_experiment",
    run_id="heart_disease_synplus_run",
)

{'data': [{'doc_id': '2',
   'kind': 'Prediction',
   'key': {'participant_id': 'worker_2',
    'project_id': 'heart_disease_synplus_project',
    'expt_id': 'heart_disease_synplus_experiment',
    'run_id': 'heart_disease_synplus_run'},
   'predict': {'statistics': {'accuracy': [0.6033519553072626,
      0.7695530726256983,
      0.8268156424581006,
      0.21508379888268156,
      0.9511173184357542],
     'roc_auc_score': [0.5027957094418363,
      0.43774135313176243,
      0.4550593940714908,
      0.5068738229755179,
      0.6336060415355569],
     'pr_auc_score': [0.38416399940506385,
      0.18633857267574722,
      0.17338011510134946,
      0.17441341113195807,
      0.07074474736182354],
     'f_score': [0.0, 0.09836065573770492, 0.03125, 0.29925187032418954, 0.0],
     'TPRs': [0.0,
      0.061224489795918366,
      0.016129032258064516,
      0.9523809523809523,
      0.0],
     'TNRs': [1.0,
      0.9525483304042179,
      0.9966216216216216,
      0.0576271186440678,
   